# VQE & Noise Mitigation

**Total Points**: 100  
**Due Date**: 26th July 2025

---

## 1. Theoretical Framework

Before diving into the code, it's essential to understand the core concepts you will be implementing.

### The Variational Quantum Eigensolver (VQE)

VQE is a hybrid quantum-classical algorithm designed to find the minimum eigenvalue of a large matrix (Hamiltonian), a problem central to quantum physics and optimization. The process works as follows:

- **Hamiltonian Mapping**:  
  The problem of finding a molecule's ground state energy is translated into finding the minimum eigenvalue of its corresponding Hamiltonian operator, $H$.  
  This Hamiltonian can be expressed as a weighted sum of Pauli operators (combinations of $I$, $X$, $Y$, $Z$).  
  For this assignment, the Hamiltonian for the Hydrogen molecule ($H_2$) is provided.

- **The Ansatz**:  
  An "ansatz" is a parameterized quantum circuit, $U(\vec{\theta})$, that prepares a guess quantum state, $|\psi(\vec{\theta})\rangle$.  
  The quality of the ansatz is crucial for the success of VQE.  
  We'll use a generic but effective hardware-efficient ansatz called `TwoLocal`.

- **Expectation Value Measurement**:  
  The quantum computer prepares the state $|\psi(\vec{\theta})\rangle$ and measures the expectation value of the Hamiltonian:  
  $$
  \langle E(\vec{\theta}) \rangle = \langle \psi(\vec{\theta}) | H | \psi(\vec{\theta}) \rangle
  $$

- **Classical Optimization**:  
  A classical optimizer (e.g., SPSA, COBYLA) uses $\langle E(\vec{\theta}) \rangle$ as its cost function.  
  The optimizer suggests a new set of parameters, $\vec{\theta}'$, with the goal of minimizing the energy.

By the **variational principle**, the lowest energy found is guaranteed to be greater than or equal to the true ground state energy.

---

### Noise in Quantum Computers

Current quantum computers are **noisy**. Environmental interactions and imperfect controls cause errors.  
A common type of error is a **bit-flip** error, where a qubit in state $|0\rangle$ flips to $|1\rangle$ (or vice-versa).  
This is equivalent to a Pauli-$X$ gate being unintentionally applied.

Such errors corrupt quantum states and lead to inaccurate results — which you’ll observe in your noisy VQE simulation.

---

### Quantum Error Correction (QEC)

QEC protects quantum information from noise by encoding a logical qubit into multiple physical qubits.  
This allows detection and correction of errors without collapsing the logical information.

#### The 3-Qubit Bit-Flip Code:

- **Encoding**:
  - Logical $|0\rangle_L \rightarrow |000\rangle$
  - Logical $|1\rangle_L \rightarrow |111\rangle$

- **Error Detection**:  
  If a bit-flip occurs (e.g., $|000\rangle \rightarrow |010\rangle$), the state leaves the valid code space.  
  Measure **stabilizers**: $Z_0Z_1$ and $Z_1Z_2$.  
  The measurement outcomes form a **syndrome** to identify the flipped qubit.

- **Correction**:  
  Based on the syndrome, apply a corrective $X$ gate to fix the error.

---

## 2. Problem Statement and Objective

Determine the **ground state energy** of the Hydrogen ($H_2$) molecule using VQE:

1. Perform the calculation in an **ideal, noise-free** environment.
2. Repeat the process on a **noisy simulator** and observe degradation.
3. Implement a **3-qubit bit-flip QEC** to protect the algorithm and recover accuracy.

You’ll use **Qiskit** to build hybrid algorithms, simulate noise, and apply error correction.

---

## 3. Assignment Tasks & Grading

All work must be completed in a **single Jupyter Notebook**.

### Part 1: Ideal VQE Simulation (20 Points)

- **Construct the Hamiltonian (5 pts)**:  
  Use `qiskit.quantum_info.SparsePauliOp` to create the 2-qubit Hamiltonian for $H_2$ at interatomic distance of 0.735 Å:

  $$
  H = (-1.052 \cdot II) + (0.398 \cdot IZ) - (0.398 \cdot ZI) - (0.011 \cdot ZZ) + (0.181 \cdot XX)
  $$

- **Set up VQE (10 pts)**:
  - Use `TwoLocal` as the ansatz
  - Choose the `SPSA` optimizer
  - Use `qiskit.primitives.Estimator` for noise-free simulation

- **Execute and Report (5 pts)**:  
  Run the algorithm and print the final computed ground state energy.

### Part 2: VQE on a Noisy Simulator (20 Points)

- **Define a Noise Model (10 pts)**:  
  Use `qiskit_aer.noise` to create a `NoiseModel`. Introduce **bit-flip** (Pauli $X$) error with probability **0.05** after each CNOT.

- **Run Noisy VQE (10 pts)**:
  - Use `qiskit_aer.primitives.Estimator`
  - Apply your noise model
  - Re-run VQE and report energy. Compare with ideal case.

### Part 3: Building the QEC Components (30 Points)

- **Encoding Circuit (10 pts)**:  
  Write a function that takes a `QuantumCircuit` and a 1-qubit `QuantumRegister` input. Add gates to encode logical qubit into 3 physical qubits using bit-flip code.

- **Syndrome Measurement Circuit (10 pts)**:  
  Add gates for a full syndrome measurement cycle using 2 ancilla qubits and a classical register.

- **Correction Circuit (10 pts)**:  
  Add gates to apply corrective $X$ operations based on syndrome outcomes.

### Part 4: Integrated QEC-VQE and Analysis (30 Points)

- **Create Protected Ansatz (20 pts)**:  
  Wrap your ansatz with QEC logic:
  - Use logical and ancilla qubits
  - Apply **encoding circuit**
  - Insert translated `TwoLocal` ansatz gates (logical $\rightarrow$ physical)
  - Apply **syndrome measurement** and **correction**
  - Decode using inverse of the encoding circuit

- **Run Protected VQE (5 pts)**:  
  Execute with noisy estimator using your protected ansatz

- **Analyze and Compare (5 pts)**:  
  Print final energy from protected run.  
  Create a **table/bar chart** to compare:
  - Ideal (noise-free)
  - Noisy
  - Error-corrected
  Add a short **paragraph analysis**.

---

## 4. Hints & Best Practices

- **Logical CNOT**: Apply 3 physical CNOTs between corresponding qubits of the logical pair:
  ```
  CNOT(A_phys_0, B_phys_0)
  CNOT(A_phys_1, B_phys_1)
  CNOT(A_phys_2, B_phys_2)
  ```

- **Modular Circuits**: Use `qc.compose()` or append multiple sub-circuits.

- **Visualization**: Use `circuit.draw('mpl')` frequently.

- **Qubit Management**: Use meaningful names for registers to avoid confusion.

---

## 5. Resources & Links

- **VQE Documentation**:  
  [`qiskit.algorithms.minimum_eigensolvers.VQE`](https://quantum.cloud.ibm.com/docs/en/migration-guides/qiskit-algorithms-module)

- **Estimator Primitives**:  
  [`qiskit.primitives.Estimator`](https://quantum.cloud.ibm.com/docs/en/api/qiskit/primitives)

- **Noise Simulation**:  
  [`Qiskit Aer`](https://qiskit.org/ecosystem/aer/)

- **Circuit Library**:  
  [`TwoLocal Ansatz`](https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.circuit.library.TwoLocal)

- **Qiskit Textbook – Quantum Error Correction**:  
  [`QEC Chapter`](https://qiskit.org/textbook/ch-error-correction/)
